# A Closer Look at Model-Based Policy Gradients

---
## Linear Quadratic Gaussian (LQG) Problems
In what follows we consider MDPs with:
1. continuous state space $\mathbf{s} \in \mathcal{S} = \mathbb{R}^n$
2. continuous action space $\mathbf{a} \in \mathcal{A} = \mathbb{R}^d$
3. finite time horizon $N \in \mathbb{N}$ and timesteps $t \in \mathcal{T} = \{0, \dots, N - 1\}$
4. time-varying linear Gaussian dynamics 
    $$
    \mathbf{s}_{t+1} \sim p(\cdot| \mathbf{s}_t, \mathbf{a}_t) = \mathcal{N}\left( \cdot ~\middle|~ \mathbf{F}_t \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix} + \mathbf{f}_t, \mathbf{\Sigma}_{t} \right)
    $$
5. time-varying quadratic costs 
    $$
    r_{t+1} = R(\mathbf{s}_t, \mathbf{a}_t) = - \tfrac{1}{2} \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix}^\intercal \mathbf{C}_t \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix} - \mathbf{c}_t^\intercal \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix}
    $$
6. Gaussian-distributed initial state 
    $$
    \mathbf{s}_0 \sim \rho = \mathcal{N}(\mathbf{\mu}_\rho, \mathbf{\Sigma}_\rho)
    $$

### Imports

In [ ]:
from __future__ import annotations

import lqsvg.envs.lqr.utils as utils
import lqsvg.torch.named as nt
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from lqsvg.envs import lqr
from lqsvg.envs.lqr.gym import LQGGenerator
from torch import Tensor
from tqdm.auto import tqdm, trange

sns.set()

### Environment generation

In what follows we consider **time-invaryant dynamics**: $\mathbf{F}_t, \mathbf{f}_t, \mathbf{\Sigma}_t = \mathbf{F}, \mathbf{f}, \mathbf{\Sigma}, \forall t\in\mathcal{T}$.

In [ ]:
generator = LQGGenerator(
    n_state=2,
    n_ctrl=2,
    horizon=10,
    seed=123,
    stationary=True,
    passive_eigval_range=(0.5, 1.5),
    controllable=True,
    transition_bias=False,
    rand_trans_cov=False,
    rand_init_cov=False,
    cost_linear=False,
    cost_cross=False,
)

In [ ]:
with nt.suppress_named_tensor_warning():
    dynamics, cost, init = generator(n_batch=None)

print(
    f"""
Dynamics:
    F: {dynamics.F.shape}, {dynamics.F.names}
    f: {dynamics.f.shape}, {dynamics.f.names}
    W: {dynamics.W.shape}, {dynamics.W.names}

Cost:
    C: {cost.C.shape}, {cost.C.names}
    c: {cost.c.shape}, {cost.c.names}
    
Initial state:
    mean: {init.mu.shape}, {init.mu.names}
    covariance: {init.sig.shape}, {init.sig.names}
"""
)

### Useful diagnostics

In [ ]:
def sys_eigvals(dynamics: lqr.LinSDynamics) -> np.ndarray:
    eigvals = utils.stationary_eigvals(dynamics)
    return eigvals

In [ ]:
def abs_eigvals(dynamics: lqr.LinSDynamics) -> np.ndarray:
    return np.abs(sys_eigvals(dynamics)).reshape((-1,))

In [ ]:
# def min_eigval_distance(dynamics: lqr.LinSDynamics) -> np.ndarray:
#     eigvals = sys_eigvals(dynamics)[..., :, np.newaxis]
#     eigvals_T = eigvals.swapaxes(-2, -1)
#     abs_diff = np.abs(eigvals - eigvals_T)
#     return np.min(abs_diff, axis=(-2, -1))

In [ ]:
def max_abs_eigval(dynamics: lqr.LinSDynamics) -> np.ndarray:
    return np.abs(sys_eigvals(dynamics)).max(axis=-1)

In [ ]:
def all_real_eigval(dynamics: lqr.LinSDynamics) -> np.ndarray:
    eigval = utils.stationary_eigvals(dynamics)
    return np.all(np.isreal(eigval), axis=-1)

In [ ]:
def rank_defficient(dynamics: lqr.LinSDynamics) -> np.ndarray:
    n_state, _, _ = lqr.dims_from_dynamics(dynamics)
    A, _ = utils.stationary_dynamics_factors(dynamics)
    return np.linalg.matrix_rank(A.numpy()) < n_state

In [ ]:
def issymmetric(dynamics: lqr.LinSDynamics) -> np.ndarray:
    A, _ = utils.stationary_dynamics_factors(dynamics)
    return np.all(torch.abs(A - nt.transpose(A)).numpy() < 1e-8, axis=(-2, -1))

### Stability of the un-actuated system

Suppose we run the system with 0 inputs for an indefinite amount of time. Will the values of $\mathbf{s}_t$ diverge? We test for this by checking that the eigenvalues $\{\lambda_i \}_{i=1}^{n}$ of $\mathbf{F_s}$, where $\mathbf{F} = [\mathbf{F_s\ F_a}]$, are all within the unit circle in the complex plane.

![](images/stable_eigvals.png)

In [ ]:
utils.isstable(dynamics)

Let's see if we spot any trends when generating transition dynamics by sampling the entries in $\mathbf{F}$ independently from a standard Normal distribution. ($\mathcal{N}(0, 1)$)

In [ ]:
def standard_normal_dynamics(gen: LQGGenerator, samples: int) -> lqr.LinSDynamics:
    dynamics = gen.make_dynamics(n_batch=samples)
    A, B = utils.stationary_dynamics_factors(dynamics)
    F = torch.cat([torch.randn_like(A), B], dim=-1).expand_as(dynamics.F)
    F = nt.horizon(F)
    return lqr.LinSDynamics(F=F, f=dynamics.f, W=dynamics.W)

In [ ]:
stable = utils.isstable(standard_normal_dynamics(generator, 10))
print(stable)

We see that a lot of the dynamics generated are unstable. Let's plot a histogram of stable vs. unstable systems.

In [ ]:
sns.histplot(data=utils.isstable(standard_normal_dynamics(generator, 500)).astype(str))
plt.show()

We quickly see that a majority of systems generated this way are unstable. We may then see how unstable by plotting a histogram of the magnitude of the largest eigenvalue.

In [ ]:
sns.histplot(data=abs_eigvals(standard_normal_dynamics(generator, 1000)))
plt.show()

In [ ]:
sns.histplot(data=max_abs_eigval(standard_normal_dynamics(generator, 1000)))
plt.show()

We can see that a lot of the generated matrices have eigenvalues with norms well above 1.

Let's also check if the trends observed above generalize to different state sizes.

In [ ]:
def hist_by_state_dim(func: callable, sampler: callable, **kwargs):
    n_samples = 1000
    scale = 1.5
    fig, ax = plt.subplots(1, figsize=[scale * 6.4, scale * 4.8])
    x = "State size"
    y = func.__name__.replace("_", " ").capitalize()
    kwargs.update(discrete=(True, False))

    dfs = []
    state_sizes = [2 + i for i in range(19)]
    for state_size in tqdm(state_sizes):
        with generator.config(n_state=state_size):
            samples = func(sampler(generator, n_samples))
            if samples.dtype == bool:
                samples = samples.astype(str)
                kwargs.update(stat="probability", discrete=(True, True))

            state_dim = np.full_like(samples, fill_value=generator.n_state, dtype=int)
            dfs += [pd.DataFrame({y: samples, x: state_dim})]

    sns.histplot(ax=ax, x=x, y=y, data=pd.concat(dfs), cbar=True, **kwargs)

    fig.suptitle(sampler.__name__.replace("_", " ").capitalize())
    plt.tight_layout()
    plt.xticks(state_sizes)
    plt.show()

In [ ]:
hist_by_state_dim(rank_defficient, standard_normal_dynamics)

hist_by_state_dim(abs_eigvals, standard_normal_dynamics)

hist_by_state_dim(max_abs_eigval, standard_normal_dynamics)

hist_by_state_dim(all_real_eigval, standard_normal_dynamics)

hist_by_state_dim(utils.isstable, standard_normal_dynamics)

hist_by_state_dim(utils.iscontrollable, standard_normal_dynamics)

hist_by_state_dim(utils.is_pbh_ctrb, standard_normal_dynamics)

### Generating stable dynamics

To have stable dynamics we generate random matrices $\mathbf{F_s}$ with eigenvalues with absolute values less or equal to 1. We start by sampling each eigenvalue independently from the uniform distribution $\mathcal{U}(-1, 1)$. Then we use the trick described in the blog post [Generate a random matrix with specified eigenvalues](https://blogs.sas.com/content/iml/2012/03/30/geneate-a-random-matrix-with-specified-eigenvalues.html) to generate a random matrix with the sampled eigenvalues.

<div class="alert alert-block alert-info">
    <b>Note:</b> for now we sample only real eigenvalues
</div>

In [ ]:
def stable_dynamics(gen: LQGGenerator, samples: int) -> lqr.LinSDynamics:
    with gen.config(passive_eigval_range=(0.0, 1.0)):
        return gen.make_dynamics(n_batch=samples)

In [ ]:
hist_by_state_dim(rank_defficient, stable_dynamics)

hist_by_state_dim(utils.isstable, stable_dynamics)

hist_by_state_dim(max_abs_eigval, stable_dynamics)

hist_by_state_dim(abs_eigvals, stable_dynamics)

hist_by_state_dim(all_real_eigval, stable_dynamics)

hist_by_state_dim(issymmetric, stable_dynamics)

### Testing controllability

Next, we build the **controllability matrix** for randomly generated systems and visualize the distribution of the controllability status.

In [ ]:
hist_by_state_dim(utils.iscontrollable, standard_normal_dynamics)

hist_by_state_dim(utils.iscontrollable, stable_dynamics)

We see that for standard normal generators the resulting systems are mostly uncontrollable as the state size increases. On the other hand, the stable dynamics generated are mostly controllable.

Let's try to generate stable, controllable systems by exploiting the consequences of the Popov-Belevitch-Hautus (PBH) test.

In [ ]:
def stable_ctrb_dynamics(gen: LQGGenerator, samples: int) -> lqr.LinSDynamics:
    with gen.config(passive_eigval_range=(0.0, 1.0), controllable=True):
        return gen.make_dynamics(n_batch=samples)

In [ ]:
def unstable_ctrb_dynamics(gen: LQGGenerator, samples: int) -> lqr.LinSDynamics:
    with gen.config(passive_eigval_range=(0.5, 1.5), controllable=True):
        return gen.make_dynamics(n_batch=samples)

In [ ]:
hist_by_state_dim(utils.iscontrollable, unstable_ctrb_dynamics)

hist_by_state_dim(utils.iscontrollable, stable_ctrb_dynamics)

Epic fail :/

### Testing Controllability via PBH

In [ ]:
# hist_by_state_dim(utils.is_pbh_ctrb, standard_normal_dynamics)

# hist_by_state_dim(utils.is_pbh_ctrb, stable_dynamics)

# hist_by_state_dim(utils.is_pbh_ctrb, stable_ctrb_dynamics)

### Testing Stabilizability

In [ ]:
# hist_by_state_dim(utils.isstabilizable, standard_normal_dynamics)

# hist_by_state_dim(utils.isstabilizable, stable_dynamics)

# hist_by_state_dim(utils.isstabilizable, stable_ctrb_dynamics)